In [16]:
import os 
import pandas as pd 
from bs4 import BeautifulSoup
from io import StringIO

In [ ]:
%pip install pandas

In [ ]:
SCORE_DIR = "data/scores"

In [ ]:
box_scores = os.listdir(SCORE_DIR)

In [ ]:
box_scores

In [ ]:
len(box_scores)

In [ ]:
box_scores = [os.path.join(SCORE_DIR, f) for f in box_scores if f.endswith(".html")]

In [ ]:
box_scores

In [10]:
def parse_html(box_score):
    with open(box_score) as f:
        html = f.read()
    soup = BeautifulSoup(html)
    [s.decompose() for s in soup.select("tr.over_header")]
    [s.decompose() for s in soup.select("tr.thead")]
    return soup 
    
    
    

In [32]:
def read_line_score(soup): #use pandas to process html table as dataframe
    html_string = str(soup)
    html_buffer = StringIO(html_string)
    line_score = pd.read_html(html_buffer, attrs={"id": "line_score"})[0]
    cols = list(line_score.columns)
    cols[0] = "team"
    cols[-1] = "total"
    line_score.columns = cols


    line_score = line_score[["team", "total"]]
    
    return line_score

In [36]:

def read_stats(soup, team, stat):
    """Reads a stats table for a given team and stat type."""
    html_string = str(soup)
    html_buffer = StringIO(html_string) 
    df = pd.read_html(html_buffer, attrs={"id": f"box-{team}-game-{stat}"}, index_col=0)[0]
    df = df.apply(pd.to_numeric, errors="coerce")
    return df  

In [99]:
base_cols = None
box_score = box_scores[0]
#for box_score in box_scores:
soup = parse_html(box_score)
#html_string = str(soup)
#html_buffer = StringIO(html_string)
#df = pd.read_html(html_buffer, attrs={"id": "line_score"})[0]
#df
#line_score = pd.read_html(html_buffer, attrs={"id": "line_score"})[0]
#cols = list(line_score.columns)
#cols[0] = "Team"
#cols[-1] = "Total"
#line_score.columns = cols

#line_score
line_score = read_line_score(soup)

teams = list(line_score["team"])
summaries = []
for team in teams:
    basic = read_stats(soup, team, "basic")
    advanced = read_stats(soup, team, "advanced")
    totals = pd.concat([basic.iloc[-1,:], advanced.iloc[-1,:]])
    totals.index = totals.index.str.lower()
    maxes = pd.concat([basic.iloc[:-1,:].max(), advanced.iloc[:-1,:].max()])
    maxes.index = maxes.index.str.lower() + "_max"
    summary = pd.concat([totals,maxes])
    if base_cols is None:
        base_cols = list(summary.index.drop_duplicates(keep="first"))
        base_cols = [b for b in base_cols if "bpm" not in b]
    summary = summary[base_cols]
    summaries.append(summary)
summary = pd.concat(summaries, axis=1).T
game = pd.concat([summary, line_score], axis=1)
game["home"] = [0,1]

In [90]:
#soup

In [91]:
#teams

In [92]:
#df

In [93]:
#advanced

In [94]:
#basic

In [95]:
#total

In [96]:
#maxes

In [97]:
#totals

In [100]:
#summary

In [101]:
game

,mp,mp,fg,fga,fg%,3p,3pa,3p%,ft,fta,...,trb%_max,ast%_max,stl%_max,blk%_max,tov%_max,usg%_max,ortg_max,drtg_max,team,total
0,240.0,240.0,36.0,88.0,0.409,8.0,32.0,0.250,19.0,25.0,...,13.7,46.5,9.9,4.5,34.7,29.9,129.0,112.0,BOS,99
1,240.0,240.0,38.0,83.0,0.458,5.0,22.0,0.227,21.0,25.0,...,19.4,43.7,3.8,6.0,31.6,27.3,138.0,107.0,CLE,102
